In [1]:
import torch
from torch.utils.data import Dataset, DataLoader

In [2]:
# load les données

fichier = open('villes.txt')
donnees = fichier.read()
villes = donnees.replace('\n', ',').split(',')
villes = [ville for ville in villes if len(ville) > 2]
villes = sorted(villes, key=len)

In [80]:
mean_len = sum(len(ville) for ville in villes) / len(villes)
mean_len

11.689774933683376

In [3]:
# création du vocabulaire

vocabulaire = sorted(list(set(''.join(villes))))
vocabulaire = ["<SOS>", "<EOS>"] + vocabulaire

# pour convertir char <-> int
char_to_int = {}
int_to_char = {}

for (c, i) in zip(vocabulaire, range(len(vocabulaire))):
    char_to_int[c] = i
    int_to_char[i] = c

In [38]:
num_sequences = len(villes)
max_len = max([len(ville) for ville in villes]) + 2 # account for <SOS> and <EOS>

X = torch.zeros((num_sequences, max_len))

for i in range(num_sequences):
    X[i] = torch.tensor([char_to_int['<SOS>']] + [char_to_int[c] for c in villes[i]] + [char_to_int['<EOS>']] + [-1] * (max_len - len(villes[i]) - 2))


"""
n_split = int(0.9*X.shape[0])

X_train = X[:n_split]
X_val = X[n_split:]
"""

n_split = int(0.9*X.shape[0])

idx_permut = torch.randperm(X.shape[0])
idx_train, _ = torch.sort(idx_permut[:n_split])
idx_val, _ = torch.sort(idx_permut[n_split:])

X_train = X[idx_train]
X_val = X[idx_val]

In [169]:
def get_batch(split, batch_size):
    # returns a batch, according to the data pipeline written in the W&B report
    data = X_train if split == 'train' else X_val

    idx_seed = torch.randint(high=data.shape[0], size=(1,)).item() #sample la ligne seed autour de laquelle on va piocher les exemples

    if split == 'train':
        idx = torch.randint(low = max(0, idx_seed - 4 * batch_size), high = min(data.shape[0], idx_seed + 4 * batch_size), size=(batch_size,)) #samples les indices du batch à produire
        
    else:
        start = max(0, idx_seed-batch_size/2)
        end =  min(data.shape[0], idx_seed+batch_size/2)

        if end-start != batch_size:
            if start == 0:
                end = end + (idx_seed-batch_size/2)
            else:
                start = start - (idx_seed-batch_size/2)


        idx = torch.arange(start=start, end=end, dtype=torch.int64)
        
    #pq 4 ? bon compromis entre assez large pour pas bcp de répétitions, assez petit pour pas bcp de padding (cf data.ipynb)
    #en moyenne sur un batch, 6.7 d'écart en max_len et min_len (donc en moyenne pour une séq., 3,3 de padding) (ça fait bcp finalement? a comparer devant la longueur d'un mot) et max_len 17.8
    #longueur moyenne d'une séq. : 11.6. donc en moyenne pour une séq., on rajoute 28% de compute inutile...
    #on aura forcement pas mal de padding sur le val car dataset comparable devant 4*batch_size

    #pour 2, on aura 3.5 d'écart, donc 1.7 de padding
    # donc 14% de compute inutile par mot

    #pour 2, on a 12% des exemples qui sont des répétitions (122 pour un batch de 1024, en moyenne)
    #pour 4, seulement 6% (67 exemples)

    #pour batch_size=512
    #pour 2, 5% de répétitions, 1.8 de disp. donc 0.9 de padding (soit 7% de compute inutile par séq.)
    #pour 4, 3% de répétitions, 3.5 de disp. donc 1.7 de padding (soit 14% de compute inutile par séq.)

    #pour privilégier des données iid, je choisi de partir sur 4 (pour avoir des batch diversifiés, au détriment d'un peu de compute inutile)
    #voir considérer plus ?
    
    #pour le val. set, 4 semble bcp trop
    #pour 4, 12% de répétitions, 15 de disp. donc 7 de padding (soit plus de 50% de compute lost)
    #c'est dommage, vu qu'avoir des batchs diversifiés n'a aucun interet pour le calcul du loss, donc on ne fait que perdre du compute
    #mais le pb avec 1 (par exemple) c'est qu'on a enormément de répétitions (25%) au sein d'un batch... donc estimation du cout totalement erronée
    #je choisis d'adopter une stratégie un peu différente: sample idx_seed, et on prends tous les index entre idx_seed-batch_size/2 et idx_seed+batch_size/2

    idx_sorted, _ = torch.sort(idx) #on les ordonne pour recuperer facilement la longueur de la plus grande seq. du batch
    print(idx_sorted.shape)

    X_batch = data[idx_sorted] #on extrait la matrice qui va produire Xb et Yb

    max_len_batch = torch.sum(torch.ne(X_batch[-1], -1)) #longueur de la plus grande seq. du batch : torch.ne(X_batch[-1], -1) crée une matrice masque, avec True si diff de -1, False si egal a -1
    print(max_len_batch)

    Xb = X_batch[:, :max_len_batch-1] #on selectionne que jusqu'a la len max - 1 (<EOS> du plus long inutile) (le reste n'est que padding)
    Yb = X_batch[:, 1:max_len_batch] #meme que Xb, mais décalé de 1 (avec le <EOS> mais sans le <SOS>)

    #Xb[Xb == 1] = -1 #on remplace le <EOS> par du padding (totalement optionnel)

    Xb = Xb.pin_memory().to('cuda', non_blocking=True)
    Yb = Yb.pin_memory().to('cuda', non_blocking=True)

    return Xb, Yb

#todo : trouver le int parfait (compter les répétitions ?)

In [170]:
Xb, Yb = get_batch('val', 1024)

torch.Size([1024])


IndexError: index 3657 is out of bounds for dimension 0 with size 3657

In [144]:
a = torch.arange(start=10, end=20).view(-1)

In [149]:
a.shape

torch.Size([10])

In [147]:
b = torch.randint(high=10, size=(10,))
b

tensor([8, 1, 3, 8, 3, 2, 5, 6, 2, 7])

In [148]:
b.shape

torch.Size([10])

In [176]:
data = X_val
batch_size = 1024

In [182]:
X_val.shape[0]

3657

In [189]:
idx_seed = torch.randint(high=data.shape[0], size=(1,)).item()
idx_seed

3415

In [193]:
start = max(0, idx_seed-batch_size/2)
end =  min(data.shape[0], idx_seed+batch_size/2)

if end-start != batch_size:
    if start == 0:
        end = end + (idx_seed-batch_size/2 - start)
    else:
        start = start - (idx_seed+batch_size/2 - end)

start, end = int(start), int(end)

start, end

(2633, 3657)

In [143]:
print(a.shape)

torch.Size([10])


In [122]:
def get_batch_len(split, batch_size):
    data = X_train if split == 'train' else X_val

    num_repet_mean = 0
    max_len_mean = 0
    disp_mean = 0
    for i in range(1000):
        idx_seed = torch.randint(high=data.shape[0], size=(1,)).item()

        idx = torch.randint(low = max(0, idx_seed - 1 * batch_size), high = min(data.shape[0], idx_seed + 1 * batch_size), size=(batch_size,))

        idx_sorted, _ = torch.sort(idx)

        X_batch = data[idx_sorted]

        min_len_batch = torch.sum(torch.ne(X_batch[0], -1))
        max_len_batch = torch.sum(torch.ne(X_batch[-1], -1))

        num_repet_mean += len(idx) - len(torch.unique(idx))
        max_len_mean += max_len_batch
        disp_mean += (max_len_batch-min_len_batch)
    
    print(num_repet_mean/1000, max_len_mean/1000, disp_mean/1000)

In [123]:
get_batch_len('val', 1024)

257.071 tensor(22.1720) tensor(13.3070)


In [124]:
257/1024

0.2509765625

In [118]:
Xb, Yb = get_batch('val', 1024)

tensor(46)


In [119]:
for i in range(Xb.shape[0]):
    nom_X = ""
    for id in Xb[i]:
        if int(id.item()) == -1:
            nom_X += "<pad>"
        else:
            nom_X += int_to_char[int(id.item())]

    nom_Y = ""
    for id in Yb[i]:
        if int(id.item()) == -1:
            nom_Y += "<pad>"
        else:
            nom_Y += int_to_char[int(id.item())]
            
    print(nom_X)
    print(nom_Y)
    print("**************************************************")

<SOS>ney<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
ney<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
**************************************************
<SOS>ney<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
ney<EOS><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>
**************************************************
<SOS>vry<EOS><pad><pad><pad><pad><pad><pad

### comment déterminer le 4 dans le sample des idx ?

In [432]:
batch_size = 1024

for i in range(50):
    idx_seed = torch.randint(high=X_train.shape[0], size=(1,)).item()
    
    idx = torch.randint(low = max(0, idx_seed - 4 * batch_size), high = min(X_train.shape[0], idx_seed + 4 * batch_size), size=(batch_size,))
    idx_sorted, _ = torch.sort(idx)

    X_batch = X_train[idx_sorted]

    min_len_batch = torch.sum(torch.ne(X_batch[0], -1))
    max_len_batch = torch.sum(torch.ne(X_batch[-1], -1))

    if(min_len_batch.item() > max_len_batch.item()):
        print("ouille")
    
    print(str(min_len_batch.item()) + " | " + str(max_len_batch.item()))
    print("***************************************************************")

8 | 10
***************************************************************
5 | 8
***************************************************************
13 | 19
***************************************************************
12 | 17
***************************************************************
5 | 9
***************************************************************
17 | 23
***************************************************************
5 | 9
***************************************************************
14 | 21
***************************************************************
12 | 18
***************************************************************
8 | 10
***************************************************************
19 | 23
***************************************************************
10 | 12
***************************************************************
9 | 11
***************************************************************
9 | 12
***************************************************************
13

In [228]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader, BatchSampler, Sampler

In [230]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

class BucketBatchSampler(BatchSampler):
    def __iter__(self):
        sorted_indices = sorted(list(range(len(self.sampler.data_source))), key=lambda i: len(self.sampler.data_source[i]))
        return iter([sorted_indices[i:i+self.batch_size] for i in range(0, len(sorted_indices), self.batch_size)])

def collate_fn(batch):
    return pad_sequence([torch.tensor(seq) for seq in batch], batch_first=True)

data = [  # example data: list of sequences
    [1, 2, 3, 4],
    [5, 6, 7],
    [8, 9],
    [10]
]
dataset = MyDataset(data)
bucket_sampler = BucketBatchSampler(Sampler(dataset), batch_size=2, drop_last=False)
dataloader = DataLoader(dataset, batch_sampler=bucket_sampler, collate_fn=collate_fn)

for batch in dataloader:
    print(batch)

AttributeError: 'Sampler' object has no attribute 'data_source'